In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import acquire as a

In [2]:
df = a.get_zillow_data()

In [6]:
df.dropna()

,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock


In [8]:
for column in df.columns:
    
    if df.dtypes[column] == object:
        
        df = df.drop_column(column)
        
df.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().